In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn import plotting
from bids import BIDSLayout
from bids.reports import BIDSReport
sys.path.append('..')
from utils.data import Subject, Block, load_participant_list

In [ ]:
base_dir = '/Users/hugofluhr/data/LH_dev'
all_sub_ids = load_participant_list(base_dir)

In [ ]:
subjects = [Subject(base_dir, 'sub-'+sub_id, include_modeling=True, include_imaging=False) for sub_id in all_sub_ids]

In [ ]:
sub = subjects[3]
learning_trials = sub.trials[sub.trials['block'].isin(['learning1', 'learning2'])]
stim2_score = len(learning_trials[learning_trials['chosen_stim'] == 5.])

In [ ]:
sub.trials.groupby('block')['chosen_stim'].value_counts()

In [ ]:
stim_rewards = {0: 0, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}
stim_frequ = {0: 0, 1: 0, 2: -1, 3: 1, 4: -1, 5: 1, 6: -1, 7: 1, 8: 0}

stimulus_counts = sub.trials.groupby('block').apply(
    lambda x: pd.DataFrame({
        'presented': pd.concat([x['left_stim'], x['right_stim']]).value_counts(),
        'chosen': x['chosen_stim'].value_counts()
    }).fillna(0).astype(int),
    include_groups=False
).reset_index()

print(stimulus_counts)

In [ ]:
same_value_trials = sub.trials[sub.trials['left_value'] == sub.trials['right_value']]

stimulus_counts_test = same_value_trials.groupby('block').apply(
    lambda x: pd.DataFrame({
        'presented': pd.concat([x['left_stim'], x['right_stim']]).value_counts(),
        'chosen': x['chosen_stim'].value_counts()
    }).fillna(0).astype(int),
    include_groups=False
).reset_index()

print(stimulus_counts_test)

In [ ]:
# Filter the data for learning1 and test blocks
learning1_counts = stimulus_counts[(stimulus_counts['block'] == 'learning1')|(stimulus_counts['block'] == 'learning2')]
test_counts = stimulus_counts_test[stimulus_counts_test['block'] == 'test']

# Create a scatter plot for each level_1 value
levels = [2, 3, 4, 5, 6, 7]
plt.figure(figsize=(10, 6))

for level in levels:
    x = learning1_counts[learning1_counts['level_1'] == level]['chosen']
    y = test_counts[test_counts['level_1'] == level]['chosen']
    plt.scatter(x, y, label=f'level_1 = {level}')

plt.xlabel('Number of times chosen in learning1')
plt.ylabel('Number of times chosen in test')
plt.title('Scatter plot of chosen stimuli in learning1 vs test')
plt.legend()
plt.show()

In [ ]:
# I think?
rare_stim = [2., 5., 6.]
frequent_stim = [2, 4, 6]

for sub in subjects:
    stim1_score = sub.trials